## This is meant to be for small experiments

In [1]:
import brian2 as b2
import matplotlib.pyplot as plt
import numpy as np
import preprocessing
import scipy as sp
%matplotlib notebook

from input_factory import create_ou_current

In [2]:
I_exc = create_ou_current(3,2000,Delta_sigma=4.0,sigma_0=10.0)
I_fs = create_ou_current(15,250,Delta_sigma=10.0,sigma_0=10.0)
I_nfs = create_ou_current(1,250,Delta_sigma=2.0,sigma_0=10.0)

In [3]:
np.savetxt('Iexc.txt',I_exc)
np.savetxt('Ifs.txt',I_fs)
np.savetxt('Infs.txt',I_nfs)

In [4]:
eqs = '''
dv/dt = -v/tau : 1
'''
tau = 4 *b2.ms
G1 = b2.NeuronGroup(10, eqs,threshold='v>4',reset=3.5)
G2 = b2.NeuronGroup(10,eqs,threshold='v>4',reset=3.5)
S = b2.Synapses(G1,G2, 'w : 1',on_pre='v+=w')
S.connect(p=1)
S.w = 2


M1 = b2.StateMonitor(G1, 'v', record=True)
M2 = b2.StateMonitor(G2, 'v', record=True)
S1 = b2.SpikeMonitor(G1)
S2 = b2.SpikeMonitor(G2)
# I_new = np.loadtxt('I.txt') * b2.namp
# I = b2.TimedArray(I_new, dt = 1.*b2.ms)

In [5]:
G_net = b2.Network(G1,G2,S)
G1.v = 4.5
G2.v = 4.5
G_net.run(15*b2.ms)

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'exact' (took 0.09s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'neurongroup', using method 'exact' (took 0.02s). [brian2.stateupdaters.base.method_choice]


BrianObjectException: Original error and traceback:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/brian2/core/network.py", line 841, in before_run
    obj.before_run(run_namespace)
  File "/anaconda3/lib/python3.6/site-packages/brian2/groups/group.py", line 1079, in before_run
    self.update_abstract_code(run_namespace=run_namespace)
  File "/anaconda3/lib/python3.6/site-packages/brian2/groups/neurongroup.py", line 308, in update_abstract_code
    raise TypeError(error_msg)
TypeError: Reset statement has to be a string, not <class 'float'>. Probably you intended to use 'v = ...'?

Error encountered with object named "neurongroup_1_resetter".
Object was created here (most recent call only, full details in debug log):
  File "<ipython-input-4-066bca9eb0de>", line 6, in <module>
    G2 = b2.NeuronGroup(10,eqs,threshold='v>4',reset=3.5)

An error occurred when preparing an object. TypeError: Reset statement has to be a string, not <class 'float'>. Proba

In [ ]:
fig = plt.figure()
plt.plot(M1.t/b2.ms, M1.v[1])
plt.xlabel('Time (ms)')
plt.ylabel('v')

In [ ]:
G_net.store('4 sec')

In [ ]:
import params